In [5]:
!pip install PyPDF2
!pip install faiss-cpu

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.3/31.3 MB 50.4 MB/s eta 0:00:00


In [6]:
import PyPDF2
import re
import numpy as np
import faiss
from sentence_transformers import SentenceTransformer
from langchain.text_splitter import RecursiveCharacterTextSplitter
from google.colab import files
from faiss import IndexFlatIP
import torch

In [7]:
ruta_pdf = '/content/drive/MyDrive/Bootcamp/IA/Principles of Horticulture.pdf'
def procesar_pdf(ruta_pdf):
    with open(ruta_pdf, 'rb') as file:
        reader = PyPDF2.PdfReader(file)
        text = " ".join([page.extract_text() for page in reader.pages])
        return re.sub(r'\s+', ' ', text)  # Limpieza básica

texto = procesar_pdf(ruta_pdf)

# Guardar el texto en un archivo .txt
with open('texto_preprocesado.txt', 'w', encoding='utf-8') as f:
    f.write(texto)

# Descargar el archivo
files.download('texto_preprocesado.txt')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [8]:
#Divide the text in chuncks
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=500,  # Número de caracteres por fragmento
    chunk_overlap=100, # Solapamiento para mantener contexto
)

chunks = text_splitter.split_text(texto)

In [9]:
chunks[123]

'the red toadstool well known in fairy tales, is a mycorrhizal fungus found mainly on spruce roots. Mycorrhizae have been shown in many species to have important roles in absorbing water and nutrients such as phosphate and nitrate. Phosphate is the least mobile major nutrient in the soil, and the plant must ‘reach out’ to ﬁnd it (see page 203). Mycorrhizae help plants to do this. An extreme case is the heather (Erica) genus which is often found growing in acidic soils, where phosphate'

In [10]:
#!pip install transformers
#!pip install sentence-transformers faiss-cpu

In [11]:
#1. Cargar modelo (se descarga una vez, luego queda en cache)
device = 'cuda' if torch.cuda.is_available() else 'cpu'
model = SentenceTransformer('all-MiniLM-L6-v2').to(device)

try:
  index = faiss.read_index("mi_indice.faiss")
except:
  chunk_embeddings = model.encode(chunks, show_progress_bar=True)
  index = IndexFlatIP(384)  # Índice para similitud coseno
  index.add(np.array(chunk_embeddings, dtype='float32'))
  faiss.write_index(index, "mi_indice.faiss")

# Búsqueda semántica
query = "What prunning methods are mentioned in the text?"
query_embedding = model.encode([query], normalize_embeddings=True)
D, I = index.search(np.array(query_embedding, dtype='float32'), k=3)  # Top 3 resultados

# Mostrar fragmentos más relevantes
for idx in I[0]:
    print(f"Similitud: {D[0][np.where(I[0] == idx)[0][0]]:.2f} - Fragmento: {chunks[idx][:100]}...")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Batches:   0%|          | 0/60 [00:00<?, ?it/s]

Similitud: 0.37 - Fragmento: Rosaceae family, are able to produce under- ground adventitious buds that grow into aerial stems or ...
Similitud: 0.36 - Fragmento: 6~4~04 5:37 PM Page 71 mature plant (formative pruning). For example, a young apple tree (called a ‘...
Similitud: 0.36 - Fragmento: a com- bination of methods. Hoeing or alternative culti- vation, particularly in spring, prevents de...


In [13]:
# Búsqueda semántica
query = "what is photosynthesis?"
query_embedding = model.encode([query], normalize_embeddings=True)
D, I = index.search(np.array(query_embedding, dtype='float32'), k=3)  # Top 3 resultados

# Mostrar fragmentos más relevantes
for idx in I[0]:
    print(f"Similitud: {D[0][np.where(I[0] == idx)[0][0]]:.2f} - Fragmento: {chunks[idx][:100]}...")

Similitud: 0.73 - Fragmento: result in an increase in size and weight, and those processes which cause the changes in the plant d...
Similitud: 0.67 - Fragmento: 54 Plant Growth 55 Light Light is a factor required in order that photosyn- thesis can occur. In any...
Similitud: 0.65 - Fragmento: in the differentorgans and tissues throughout the plant.chap-06.qxd 6~4~04 5:36 PM Page 53 A summary...


In [12]:
# Save chunks, embeddings and index
ruta_base = '/content/drive/MyDrive/Bootcamp/IA/'
np.save(f"{ruta_base}horticultura_chunks.npy", np.array(chunks, dtype=object))  # Guarda los chunks
np.save(f"{ruta_base}horticultura_embeddings.npy", chunk_embeddings)  # Guarda los embeddings

index = faiss.IndexFlatIP(384)
index.add(chunk_embeddings.astype('float32'))
faiss.write_index(index, f"{ruta_base}horticultura_index.faiss")

print("✅ Procesamiento completado. Archivos guardados:")
print("- horticultura_chunks.npy (textos)")
print("- horticultura_embeddings.npy (vectores)")
print("- horticultura_index.faiss (índice de búsqueda)")

✅ Procesamiento completado. Archivos guardados:
- horticultura_chunks.npy (textos)
- horticultura_embeddings.npy (vectores)
- horticultura_index.faiss (índice de búsqueda)
